In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
priority_data = pd.read_csv('/kaggle/input/priority-1stversion/priority_data_1st-version.csv', delimiter = ';')

In [ ]:
from sklearn.model_selection import train_test_split


# Separate target from predictors
y = priority_data.Priority
X = priority_data.drop(['Priority'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 12 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()

X_valid = X_valid_full[my_cols].copy()



In [ ]:
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in categorical_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])


In [ ]:


from sklearn.impute import SimpleImputer

# Imputation for label
my_imputer = SimpleImputer()
label_imputed_X_train = pd.DataFrame(my_imputer.fit_transform(label_X_train))
label_imputed_X_valid = pd.DataFrame(my_imputer.transform(label_X_valid))

# Imputation removed column names; put them back
label_imputed_X_train.columns = label_X_train.columns
label_imputed_X_valid.columns = label_X_valid.columns



In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

#model = XGBRegressor(n_estimators = 20)
#model = LGBMRegressor(random_state=5)
model = RandomForestRegressor(n_estimators=400, random_state=0)
#model = DecisionTreeRegressor(random_state=1)

In [ ]:

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


model.fit(label_imputed_X_train, y_train)

# Preprocessing of validation data, get predictions
preds = model.predict(label_imputed_X_valid)
errors = abs(preds - y_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

